In [ ]:
import torch
import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt
from Dataset import Dataset
from torch.utils.data import DataLoader

In [ ]:
x_data = np.linspace(-100, 100, 30000)
y_data = np.sin(x_data) / x_data

In [ ]:
dataset = Dataset(torch.tensor(x_data).float(), torch.tensor(y_data).float())

trainsize, testsize = int(0.9*len(dataset)), len(dataset)-int(0.9*len(dataset))
trainset, testset = torch.utils.data.random_split(dataset, [trainsize, testsize], generator=torch.Generator().manual_seed(42))

trainloader = DataLoader(trainset, batch_size=128, shuffle=True)
testloader = DataLoader(testset, batch_size=128, shuffle=False)

activation = nn.Sigmoid()

In [ ]:
class KELM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = activation(x)
        x = self.fc2(x)
        return x

model = KELM(1, 1000, 1)

In [ ]:
V = activation(model.fc1.bias.data.reshape(-1, 1) + model.fc1.weight.data @ trainset[:][0].reshape(1, -1))
H = torch.concatenate([torch.ones(1, len(trainset)), V], dim=0).T
Y = trainset[:][1].reshape(1, -1).T

In [ ]:
def f(x, gamma=0.1):
    return torch.exp(-gamma*torch.pow(torch.tensor(x).reshape(-1, 1).float()-trainset[:][0].reshape(1, -1), 2)) @ torch.linalg.inv(H @ H.T) @ Y

In [ ]:
plt.plot(x_data, y_data)
plt.plot(x_data, f(x_data), '--')
plt.legend(['Dane', 'Model'], loc='upper right')
plt.grid()
plt.show()